In [222]:
from numpy import cos, sin, radians, pi, sqrt, interp, log10, log, asarray, abs
import pandas as pd

Informações enunciado

In [223]:
m = 4                  #modulo
Np, Ng= 20, 60         #número de dentes 
theta = 20             #angulo de pressão
ωp = 1200              #rotação pinhão
sy = 65                #MPa       #ignora
n = 1                  #coeficiente de segurança   #ignora
η = 0                  #eficiencia  #%      #ignora
v = 0.29               #coeficiente de poisson
b = 41                 #largura da face 6 a 16*m  
DE = 47                #diametro do eixo #mm

In [224]:
Ep = Eg = 205*10**3     #MPa
vp = vg = v 
S = 300   #mm
S1 = 53   #mm
cte = [ 2.47*10**(-1),
        0.657*10**(-3),
       -1.186*10**(-7) ]

Av = 9                 #assumindo indice de qualidade 9

In [225]:
ZW = 1    #fator de razão de dureza para resistencia ao crateramento    #O valor de ZW para o pinhão é fixado em 1,0.
Y0 = 1    #fator de temperatura     #1 p/ até 120C
YJ = 0.335   #grafico em função de N

In [226]:
Lp = Lg = 41                #meses
qp = qg = 1                 #p/ rotação em 1 direção só
confp = confg = 99          #confiabilidade #%
ft = [3.32, 1.89]           #tabelado em função do número de dentes

In [227]:
#r = [ft[0]*m, ft[1]*m] 

In [228]:
#r[0], r[1]

In [229]:
def de(N):
    return m*( N + 2 )                        #d_externo

def dr(N):
    return m*( N - 2.5 )                      #d_raiz

def dp(N):
    return m*N                                #d_primitivo  
         
def db(N):
    return cos(radians(theta))*dp(N)          #d_base

def V(N, ω):
    return (pi*ω/30)*(dp(N)/2000)             #velocidade linear

def KV(N, ω):
    return (6.1 + V(N, ω))/6.1                #fator de velocidade

In [230]:
p = m*pi                                 #passo circular
e = p/2                                  #espessura do dente
r = 0.3*m                                #raio do filete 
i = Ng/Np                                #relação de engrenamento
f = 0.3*m                                #cordão raiz do dente (ou raio do filete)
d_centros = ( dp(Np) + dp(Ng) )/2        #Distancia entre centros 

In [231]:
##FATOR DE FORMA##
def Y(N):
    N_l = [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 26, 28, 30, 34, 38, 43, 50, 60, 75, 100, 150, 300, 400]
    Y_l = [0.2450, 0.2610, 0.2770, 0.2900, 0.2960, 0.3030, 0.3090, 0.3140, 0.3220, 0.3280, 0.3310, 0.3370, 0.3460, 0.3530, 0.3590, 0.3710, 0.3840, 0.3970, 0.4090, 0.4220, 0.4350, 0.4470, 0.4600, 0.4720, 0.4800]
    return interp(N, N_l, Y_l)

In [232]:
σmax = sy/n                                                                                  #tensão máxima
ωg = ωp/i                                                                                    #rotação na coroa

#def wt(N, ω):
#    return ( sy*b*m*Y(N) ) / ( n*KV(N, ω) )                                                  #carga tangecial

def wt(N, ω):
    return 2050                                                 #carga tangecial

def T(N, ω):                                                                                 #torque
    return wt(N, ω)*(dp(N)/2000)

Pp, Pg = wt(N=Np, ω=ωp)*V(N=Np, ω=ωp), ( T(N=Ng, ω=ωg) * ωg )*(100 - η)/ ( 100 * 9.55 )      #potencia


In [234]:
## CONSTANTES PARA CALCULAR O FATOR DINAMICO kv ##
B = 0.25 * (Av - 5)**(2/3)
C = 3.5637 + 3.9914*( 1 - B )

In [235]:
Khma = cte[0] + cte[1]*b + cte[2]*(b**2)         #fator de alinhamento de engrenamento
Khe = 1  
if S1/S<0.175:                                        #fator de ajuste
    Khpm = 1
if S1/S>=0.175:
    Khpm = 1.1                                                 #fator de carga de deflexão
Khmc = 1                                         #fator de formato da face do dente

def Khpf(N):
    return ( b/(10*dp(N)) ) - 0.0375 + 0.000492*b   #fator de proporção do pinhão

In [236]:
def Ko():
    return 1      #maquina acionada = uniforme; fonte de alimentação=uniforme                                          #fator de sobrecarga      #motor elétrico que aciona uma bomba centrífuga de baixa velocidade

def Kh(N):
    return 1 + Khmc * ( Khpf(N) * Khpm + Khma * Khe)    #fator de distribuição de carga

def kv(N, ω):
    return ( C / (C + sqrt(V(N, ω)) ) )**(-B)         #fator dinâmico

#def Ks():
#    return 1                                             #fator de tamanho

if b>=2.79 and b<=51:
    def Ks(N):
        return 1/( 1.18337*(b*m*sqrt(Y(N)))**(-0.0535) )   #p/ b de 2,79 a 51mm

if b>=52 and b<=254:
    def Ks(N):
        return 1/( 1.4098*(b*m*sqrt(Y(N)))**(-0.0785) )   #p/ b de 52 a 254mm


In [238]:
def Zr():
    return 1                                                                   #fator de condição de superfície

def Ze():
    return sqrt( 1/( pi * ( ( (1 - vp**2) / Ep) + (( 1 - vg**2 ) / Eg) ) ) )   #coeficiente de elasticidade

In [240]:
def C_c():
    return cos(radians(theta)) * sin(radians(theta)) * i / ( 2*(i+1) )   #fator de curvatura na linha primitiva

def C_1(N):
     return N*sin(radians(theta))/2

def C_2(N):
    return C_1(N)*i

def C_3(N):
    return pi*cos(radians(theta))

def C_4(N):
     return 0.5 * ( sqrt( ((N + 2)**2) - (N*cos( radians(theta)))**2 ) - sqrt( (N**2) - (N*cos( radians(theta) ))**2 ) )

def C_x(N):
    return (C_1(N) - C_3(N) + C_4(N))*(C_2(N) + C_3(N) - C_4(N)) / ( C_1(N)*C_2(N) )  # fator para ajuste da altura específica do LPSTC (ponto extremo inferior de contato de um dente)

In [241]:
## FATOR GEOMÉTRICO PARA RESISTÊNCIA AO CRATERAMENTO ##

def Zi(N):
    return C_c()*C_x(N)                

In [242]:
## TENSÃO DE CONTATO ##

def 𝜎H(N, ω):
    return 𝑍e()*sqrt( ( wt(N, ω) * 𝐾o() *  kv(N, ω) * 𝐾s(N) * 𝐾h(N) * 𝑍r() ) / (dp(N) * 𝑏 * 𝑍i(N)) )  

In [243]:
## FATOR DE CONFIABILIDADE ##
## CONSIDERANDO A TABELA ##

def YZ(conf):
    data_yz = {
        'confiabilidade': [99.99, 99.9, 99, 90, 50],
        'Yz': [1.5, 1.25, 1, 0.85, 0.70]
    }
    df_yz = pd.DataFrame(data_yz)
    return float(df_yz.loc[df_yz['confiabilidade']==float(conf)]['Yz'])

## CONSIDERANDO A EQUAÇÃO ##

#def YZ(conf):
#    return 0.658 - 0.0759*log(1-conf/100)   #0,5<=conf<=0,99
#
#def YZ(conf):
#    return 0.5 - 0.109*log(1-conf/100)      #0,99<=conf<=0,9999

In [245]:
def nL(L, ω, q):
    return 60*L*ω*q*720         #número de ciclos de tensão

def ZN(L, ω, q):
    return 1.4488*nL(L, ω, q)**(-0.023)    #fator do ciclagem de tensão para resistência ao crateramento
  
def HB(N, ω, L, q, conf):
    return ( ( 𝜎H(N, ω)*Y0*YZ(conf) / ( ZN(L, ω, q)*ZW ) ) - 200 ) / 2.22      #dureza brinel preliminar #considerando SH = 1

In [246]:
def brinell(value):
    data_brinell = [111,116,121,126,131,137,143,149,156,163,167,170,174,179,183,187,192,197,201,207,212,217,223,229,235,
     241,248,255,262,269,277,285,293,302,311,321,331,341,352,363,375,388,401,415,429,444,461,477,495,514,534,555,578,601,
     627,653,682,712,745]

    def find_nearest(array, value):
        array = asarray(array)
        idx = (abs(array - value)).argmin() 
        return array[idx]

    if find_nearest(data_brinell, value)>=value:
        idx = find_nearest(data_brinell, value)
    else: 
        array = asarray(data_brinell)
        idx = array[(abs(array - value)).argmin()  + 1]

    return idx       #dureza brinel (entra o valor preliminar HB e retorna o valor mais próximo (acima) dos valores tabelados de dureza)

In [247]:
def 𝜎HB(N, ω, L, q, conf):
    return brinell(HB(N, ω, L, q, conf))*2.22 + 200          #Número ao contato permitido

def SH(N, ω, L, q, conf):
    return 𝜎HB(N, ω, L, q, conf)*ZN(L, ω, q)*ZW/(𝜎H(N, ω)*Y0*YZ(conf))    #fator de segurança AGMA ao contato

In [248]:
A1 = [6,8,10,12,17,22,30,38,44,50,58,65,75,85,95,110,130,150,170,200,230,260,290,330,380,400]
t2 = [1,1.4,1.8,2.3,2.8,3.3,3.3,3.3,3.8,4.3,4.4,4.9,5.4,5.4,6.4,7.4,8.4,9.4,10.4,11.4,12.4,12.4,14.4,15.4,17.4,19.5]
A2 = [8,10,12,17,22,30,38,44,50,58,65,75,85,95,110,130,150,170,200,230,260,290,330,380,440,500]
t1 = [1.2,1.8,2.5,3,3.5,4,5,5,5.5,6,7,7.5,9,9,10,11,12,13,15,17,20,20,22,25,28,31]

In [249]:
for c in range(len(A1)):
    if DE<=A2[c] and DE>A1[c]:
        t = [t1[c], t2[c]]

In [250]:
def ht(N):
    return (de(N) - dr(N))/2

In [251]:
def tr(N):
    return (dr(N)/2) - (DE/2) + t[1]

In [252]:
def mb(N):
    return tr(N)/ht(N)

In [253]:
def KB(N):
    if mb(N)<1.2:
        return 1.6*log(2.242/mb(N))
    if mb(N)>=1.2:
        return 1

In [254]:
def 𝜎F(N, ω):
    return wt(N, ω)*Ko()*kv(N, ω)*Ks(N)*Kh(N)*KB(N)/(b*m*YJ)

In [255]:
def YN(L, ω, q):
    return 1.3558*nL(L, ω, q)**(-0.0178)

In [256]:
def 𝜎FP(N, ω, L, q, conf):
    return 0.533*brinell(HB(N, ω, L, q, conf)) + 88.3

In [257]:
def SF(N, ω, L, q, conf):
    return 𝜎FP(N, ω, L, q, conf)*YN(L, ω, q)/(𝜎F(N, ω)*YZ(conf)*Y0)

In [259]:
Dep, Deg = round(de(Np), 2) , round(de(Ng), 2)
Drp, Drg = round(dr(Np), 2) , round(dr(Ng), 2)
Dpp, Dpg = round(dp(Np), 2) , round(dp(Ng), 2)
Dbp, Dbg = round(db(Np), 2) , round(db(Ng), 2)
Vlp, Vlg = round(V(N=Np, ω=ωp), 2) , round(V(N=Ng, ω=ωg), 2)
σmaxp, σmaxg = round(σmax, 2), round(σmax, 2)
KVp, KVg = round(KV(N=Np, ω=ωp), 2) , round(KV(N=Np, ω=ωp), 2)
Yp, Yg = round(Y(Np), 2) , round(Y(Ng), 2)
wtp, wtg = round(wt(N=Np, ω=ωp), 2) , round(wt(N=Ng, ω=ωg), 2)
Tp, Tg = round( T( N=Np, ω=ωp ), 2 ) , round( T( N=Ng, ω=ωg ), 2 )
𝜎Hp, 𝜎Hg = round(𝜎H(N=Np, ω=ωp), 2), round(𝜎H(N=Ng, ω=ωg), 2)
Zep, Zeg = round(Ze(), 2), round(Ze(), 2)
Kop, Kog = round(Ko(), 2), round(Ko(), 2)
kvp, kvg = round( kv(N=Np, ω=ωp), 2 ), round( kv(N=Ng, ω=ωg), 2 )
Ksp, Ksg = round(Ks(N=Np), 2), round(Ks(N=Ng), 2)
Khp, Khg = round(Kh(dp(Np)), 2), round(Kh(dp(Ng)), 2)
Zip, Zig = round(Zi(Np), 2), round(Zi(Ng), 2)
ZNp, ZNg = round(ZN(L=Lp, ω=ωp, q = qp), 2), round(ZN(L=Lg, ω=ωg, q = qg), 2)
HBp = HBg  = round(brinell(HB(N=Np, ω=ωp, L=Lp, q=qp, conf=confp)), 2)
𝜎HBp = 𝜎HBg = round(𝜎HB(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2)
SHp = SHg = round(SH(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2)
Zrp = Zrg = Zr()
YZp, YZg = round(YZ(conf=confp), 2), round(YZ(conf=confg),2)
htp, htg = round(ht(Np), 2),round(ht(Ng), 2)
KBp, KBg = round(KB(Np), 2), round(KB(Ng), 2)
𝜎FPp, 𝜎FPg = round(𝜎FP(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2), round(𝜎FP(N=Ng, ω=ωg, L=Lg, q=qg, conf=confg), 2)
SFp, SFg = round(SF(N=Np, ω=ωp, L=Lp, q=qp, conf=confp), 2), round(SF(N=Ng, ω=ωg, L=Lg, q=qg, conf=confg), 2)
YNp, YNg = round(YN(L=Lp, ω=ωp, q=qp), 2), round(YN(L=Lg, ω=ωg, q=qg), 2)
𝜎Fp, 𝜎Fg = round(𝜎F(N=Np, ω=ωp), 2), round(𝜎F(N=Ng, ω=ωg),2)
trp, trg = round(tr(N=Np), 2), round(tr(N=Ng), 2)

In [260]:
data = {
    '': ['Pinhão', 'Coroa'],
'Torque [N-m]': [ Tp, Tg ],
'Raio filete (cordão raiz do dente) [mm]': [ round(f, 2) , '-' ],
'Diâmetro primitivo [mm]': [ Dpp, Dpg ],
'Diâmetro externo [mm]': [ Dep, Deg ],
'Diâmetro raiz [mm]': [ Drp, Drg ],
'Potência [W]': [ round(Pp, 2) , round(Pg, 2) ],
'Velocidade linear [m/s]': [ Vlp, Vlg ],
'Relação de transmissão': [ round(i, 2) , '-' ],
'Rotação [rpm]': [ round(ωp ,2) , round(ωg, 2) ],
'Distância de centros [mm]': [ round(d_centros, 2) , '-' ],
'fator de sobrecarga (Ko):': [ Kop, '-' ],
'fator dinâmico (Kv):': [ kvp, '-' ],
'fator de tamanho (Ks):': [ Ksp, '-' ],
'fator de distribuição de carga (Kh):': [ Khp, '-' ],
'fator de condição de superfície (Zr):': [ Zrp, '-' ],
'fator geométrico para resistência ao crateramento (Zi):': [ Zip, '-' ],
'tensão de contato AGMA (𝜎H): [MPa]': [ 𝜎Hp, '-' ],
'fator de ciclagem de tensão para resistência ao crateramento (Zn):': [ ZNp, '-' ],
'fator de razão de dureza para resistência ao crateramento (ZW):': [ ZW, '-' ],
'fator de temperatura (Y0):': [ Y0, '-' ],
'fator de confiabilidade (YZ):': [ YZp, '-' ],
'dureza Brinell selecionada (já ajustada): [HB]': [ HBp, '-' ],
'número de resistência ao contato permitido (𝜎HB): [MPa]': [ 𝜎HBp,'- '],
'fator de segurança ao crateramento:': [ SHp, '-' ],
'Desenho técnico, apenas do pinhão:': ['-', '-']
}

In [261]:
df = pd.DataFrame(data)

In [262]:
df_transposed = df.T

In [263]:
df_transposed

,0,1
,Pinhão,Coroa
Torque [N-m],82.0,246.0
Raio filete (cordão raiz do dente) [mm],1.2,-
Diâmetro primitivo [mm],80,240
Diâmetro externo [mm],88,248
Diâmetro raiz [mm],70.0,230.0
Potência [W],10304.42,10303.66
Velocidade linear [m/s],5.03,5.03
Relação de transmissão,3.0,-
Rotação [rpm],1200.0,400.0


In [265]:
#df_transposed.to_excel('calculos.xlsx')